In [40]:
import json
import pandas as pd
import requests
import os
import numpy as np
import math

from pathlib import Path
from typing import Union
from collections import defaultdict
from sqlalchemy import create_engine


In [16]:
# see website for information:
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/

api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
               'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
               'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Variable to store the total number of electric vehicle stations in the US
total_stations_us = 0

# Dictionary to store the totals for each state
state_totals = {}

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the total number of electric vehicle stations for the specified state
        total_stations = data.get('total_results', 0)

        # Increment the total stations in the US variable
        total_stations_us += total_stations

        # Store the result in the dictionary
        state_totals[state_code] = total_stations

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Sort the dictionary by state in decreasing order based on total stations
sorted_totals = dict(sorted(state_totals.items(), key=lambda item: item[1], reverse=True))

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the sorted totals to a JSON file in the 'data_query' folder
state_data_file_path = os.path.join(output_folder, 'electric_stations_by_state.json')
with open(state_data_file_path, 'w', encoding='utf-8') as state_data_file:
    json.dump(sorted_totals, state_data_file, ensure_ascii=False, indent=4)

# Print the total number of electric vehicle stations in the US
print(f'Total electric vehicle stations in the US for fuel type ELEC: {total_stations_us}')

print(f'The sorted totals have been saved to: {state_data_file_path}')


Total electric vehicle stations in the US for fuel type ELEC: 79633
The sorted totals have been saved to: data_query/electric_stations_by_state.json


In [18]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# Set up parameters for the API request
params = {
    'fuel_type_code': 'ELEC',
    'country': 'US',
    'restricted_access': 'false',
    'api_key': api_key
}

# Make the API request and convert the response to a DataFrame
response = requests.get(base_url, params=params)
data = response.json()

if response.status_code == 200:
    total_results = data.get('total_results', 0)
    fuel_stations_df = pd.json_normalize(data['fuel_stations'])
else:
    print(f'Error: {response.status_code} - {response.text}')
    exit()

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the DataFrame to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'fuel_stations_dataframe.json')
fuel_stations_df.to_json(output_file_path, orient='records', lines=True)

print(f'The fuel station data has been saved to: {output_file_path}')
print(f'Total results: {total_results}')


The fuel station data has been saved to: data_query/fuel_stations_dataframe.json
Total results: 80110


In [20]:
# Keep only the specified fields in the DataFrame
selected_fields = ['id', 'station_name', 'street_address', 'city', 'state', 'zip',
                   'latitude', 'longitude', 'access_code', 'ev_network', 'ev_connector_types', 'status_code',
                   'ev_pricing', 'geocode_status', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num']
fuel_stations_df = fuel_stations_df[selected_fields]

# Save the DataFrame with the selected fields to a new JSON file
selected_fields_output_file_path = os.path.join(output_folder, 'fuel_stations_selected_fields.json')
fuel_stations_df.to_json(selected_fields_output_file_path, orient='records', lines=True)

print(f'The fuel station data with selected fields has been saved to: {selected_fields_output_file_path}')

# Get the total count of each unique value in the 'state' field
total_results_by_state = fuel_stations_df['state'].value_counts()

print(f'Total results by state:\n{total_results_by_state}')



The fuel station data with selected fields has been saved to: data_query/fuel_stations_selected_fields.json
Total results by state:
state
CA    19309
NY     4374
TX     4163
FL     4046
MA     3200
WA     2550
CO     2512
GA     2291
IL     2289
PA     2186
OH     2036
NC     2021
MD     2008
MI     1916
VA     1868
MN     1563
MO     1546
AZ     1465
NJ     1441
OR     1436
TN     1140
IA     1099
UT     1022
WI     1012
IN      947
CT      931
SC      749
KS      723
OK      707
NV      686
AL      581
ME      511
AR      500
KY      441
DC      440
HI      434
NE      432
VT      414
NM      385
LA      371
RI      351
NH      295
MS      255
DE      248
ID      247
WV      214
SD      198
MT      158
ND      154
WY      142
AK       66
PR       22
KA        1
Name: count, dtype: int64


In [35]:
# Load the DataFrame from the 'fuel_stations_selected_fields' file
file_path = 'data_query/fuel_stations_selected_fields.json'
fuel_stations_df = pd.read_json(file_path, orient='records', lines=True)

# Create DataFrame
station_df = fuel_stations_df.copy()

# Data Cleaning
station_df = station_df.astype({
    'id': 'Int64',
    'latitude': 'float64',
    'longitude': 'float64',
    'ev_level1_evse_num': 'Int32',
    'ev_level2_evse_num': 'Int32',
    'ev_dc_fast_num': 'Int32'
}).replace("", None)  # Remove empty strings

# Display the cleaned DataFrame
station_df.head()


,id,station_name,street_address,city,state,zip,latitude,longitude,access_code,ev_network,ev_connector_types,status_code,ev_pricing,geocode_status,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num
0,38522,Denver International Airport - Canopy Airport ...,8100 Tower Rd,Commerce City,CO,80022,39.843886,-104.771138,public,Non-Networked,"[CHADEMO, J1772, NEMA515]",E,Free,GPS,18,9,1
1,77347,Madras - Dairy Queen,483 SE 5th St,Madras,OR,97741,44.628842,-121.128920,public,EVCS,"[CHADEMO, J1772, J1772COMBO]",E,DC Fast: $7.50 per session or $19.99 for month...,200-8,1,1,2
2,223284,Scranton Motors,777 Talcottville Rd,Vernon,CT,6066,41.863200,-72.482140,public,LIVINGSTON,"[CHADEMO, J1772, J1772COMBO, NEMA515]",E,$0.5 per kWh,GPS,1,1,1


In [36]:
station_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  3 non-null      Int64  
 1   station_name        3 non-null      object 
 2   street_address      3 non-null      object 
 3   city                3 non-null      object 
 4   state               3 non-null      object 
 5   zip                 3 non-null      int64  
 6   latitude            3 non-null      float64
 7   longitude           3 non-null      float64
 8   access_code         3 non-null      object 
 9   ev_network          3 non-null      object 
 10  ev_connector_types  3 non-null      object 
 11  status_code         3 non-null      object 
 12  ev_pricing          3 non-null      object 
 13  geocode_status      3 non-null      object 
 14  ev_level1_evse_num  3 non-null      Int32  
 15  ev_level2_evse_num  3 non-null      Int32  
 16  ev_dc_fast_n

In [37]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# Selected fields to retrieve
selected_fields = ['id', 'station_name', 'street_address', 'city', 'state', 'zip',
                   'latitude', 'longitude', 'access_code', 'ev_network', 'ev_connector_types', 'status_code',
                   'ev_pricing', 'geocode_status', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num']

# List of states to filter
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
               'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
               'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Set up parameters for the API request
params = {
    'fuel_type_code': 'ELEC',
    'country': 'US',
    'restricted_access': 'false',
    'api_key': api_key
}

# List to store detailed information for stations meeting the criteria
station_data_by_state = {}

# Make API requests for each state and filter based on state codes
for state_code in state_codes:
    params['state'] = state_code  # Include state code in the parameters

    # Make the API request
    response = requests.get(base_url, params=params)
    response.raise_for_status()

    # Convert the response to a DataFrame with selected fields
    data = response.json()
    if response.status_code == 200:
        stations_df = pd.json_normalize(data['fuel_stations'], sep='_')[selected_fields]
        station_data_by_state[state_code] = stations_df.to_dict(orient='records')
    else:
        print(f'Error for {state_code}: {response.status_code} - {response.text}')

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the station data by state to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'fuel_stations_by_state.json')
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(station_data_by_state, output_file, ensure_ascii=False, indent=4)

print(f'The fuel station data organized by station and state has been saved to: {output_file_path}')


The fuel station data organized by station and state has been saved to: data_query/fuel_stations_by_state.json


In [54]:
import pandas as pd
import json

# Load the JSON data
json_file_path = 'data_query/fuel_stations_by_state.json'
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    station_data_by_state = json.load(json_file)

# Create an empty list to store dictionaries
all_dicts = []

# Function to handle NaN during integer conversion
convert_to_int = lambda value: int(value) if pd.notna(value) else 0

# Iterate through the data
for state_code, stations in station_data_by_state.items():
    for station in stations:
        # Perform data cleaning and type conversion
        cleaned_station = {
            'id': convert_to_int(station.get('id', 0)),
            'station_name': station.get('station_name', ''),
            'street_address': station.get('street_address', ''),
            'city': station.get('city', ''),
            'state': station.get('state', ''),
            'zip': station.get('zip', ''),
            'latitude': float(station.get('latitude', 0.0) or 0.0),
            'longitude': float(station.get('longitude', 0.0) or 0.0),
            'access_code': station.get('access_code', ''),
            'ev_network': station.get('ev_network', ''),
            'ev_connector_types': station.get('ev_connector_types', ''),
            'status_code': station.get('status_code', ''),
            'ev_pricing': station.get('ev_pricing', ''),
            'geocode_status': station.get('geocode_status', ''),
            'ev_level1_evse_num': convert_to_int(station.get('ev_level1_evse_num')),
            'ev_level2_evse_num': convert_to_int(station.get('ev_level2_evse_num')),
            'ev_dc_fast_num': convert_to_int(station.get('ev_dc_fast_num'))
        }

        # Append the cleaned station dictionary to the list
        all_dicts.append(cleaned_station)

# Create a DataFrame from the list of dictionaries
fuel_stations_df = pd.DataFrame(all_dicts)

# Now, 'fuel_stations_df' contains the cleaned and converted data


In [ ]:
import sqlite3

# SQLite database file path
db_file_path = 'fuel_stations.db'

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Create a cursor object
cursor = conn.cursor()

# Create a table for fuel stations
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fuel_stations (
        id INTEGER PRIMARY KEY,
        station_name TEXT,
        street_address TEXT,
        city TEXT,
        state TEXT,
        zip TEXT,
        latitude REAL,
        longitude REAL,
        access_code TEXT,
        ev_network TEXT,
        ev_connector_types TEXT,
        status_code TEXT,
        ev_pricing TEXT,
        geocode_status TEXT,
        ev_level1_evse_num INTEGER,
        ev_level2_evse_num INTEGER,
        ev_dc_fast_num INTEGER
    )
''')

# Insert data into the fuel_stations table
for index, row in fuel_stations_df.iterrows():
    values = tuple(row)
    cursor.execute('''
        INSERT INTO fuel_stations VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', values)

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f'Data has been successfully loaded into the SQLite database: {db_file_path}')
